## law_1.docx, law_2.docx
- pinecone store 저장
    - index name: 임의
- RetrievalQA 구현
    - prompt: rlm/rag-prompt
    - 질문: 전세사기


In [2]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone


## 환경변수 읽어오기
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

## 문서 읽고 분할 ##################################
## 문서 파일 목록
# doc_paths = ['law_1.docx', 'law_2.docx']

# ## 문서 로드(읽어오기)
# documents = []

# for path in doc_paths:
#     loader = Docx2txtLoader(path)
#     documents.extend(loader.load())

# ## 청크 분할 설정
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1500,
#     chunk_overlap=200,
# )

# ## 문서 분할
# document_list = text_splitter.split_documents(documents)


## 임베딩 -> 벡터 스토어(데이터베이스)에 저장
## 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'law-index'

## 파인콘: 저장
# database =  PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name,
# )

## 저장된 인덱스 가져오기
## [방법 1]
# database = PineconeVectorStore(
#     index=pc.Index(index_name),
#     embedding=embedding,
# )

## [방법 2]
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding,
)

## RetrievalQA
llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

qa_chain = (
    {
        'context': database.as_retriever() | format_docs,
        'question': RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# qa_chain.invoke('전세사기피해자 대상을 알려주세요.')
qa_chain.invoke('전세사기피해자 임대인을 알려주세요.')

'전세사기피해자의 임대인은 특별법에 따라 단순한 임대인뿐만 아니라 임대인의 대리인, 임대인의 의뢰를 받은 공인중개사, 임대인을 위해 임차인을 모집하는 자, 다수 임대인 배후의 동일인 또는 그들이 지배하는 조직 등을 포함합니다. 따라서 전세사기피해자와 관련된 임대인은 매우 넓은 범위의 이해관계자를 포함할 수 있습니다.'

In [3]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


## 환경변수 읽어오기 ############################################
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

## 벡터 스토어(데이터베이스)에서 인덱스 가져오기 ###############
## 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'law-index'

## 저장된 인덱스 가져오기
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding,
)

## RetrievalQA ##################################################
llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

qa_chain = (
    {
        'context': database.as_retriever() | format_docs,
        'question': RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# qa_chain.invoke('전세사기피해자 대상을 알려주세요.')
qa_chain.invoke('전세사기피해자 임대인을 알려주세요.')

'전세사기피해자법에 따르면 전세사기 피해자 지원을 위한 임대인은 임대인 또는 임대인의 대리인, 공인중개사, 임대인을 위해 임차인을 모집하는 자, 다수 임대인의 배후에 있는 동일인 및 그 동일인이나 조직을 배후에 둔 다수의 임대인을 포함합니다. 이러한 정의는 전세사기를 막고 피해자 지원을 강화하기 위한 법적 정의에 해당합니다.'